In [1]:
import pyxem as pxm
import pymatgen as pmg
import numpy as np
from pyxem.generators.indexation_generator import IndexationGenerator
from pyxem.utils.plot import generate_marker_inputs_from_peaks
from pyxem.utils.sim_utils import peaks_from_best_template
import hyperspy.api as hs
%matplotlib tk

In [2]:
if True:
    sample = pxm.load('/home/pc494/Documents/data/cBN/160401_MultipleZoneAxes/160401_03.blo')
    #sample = sample.inav[50:-60,15:130]
    
half_side_length = (512/2)

In [3]:
## get an undeformed section
dp = pxm.ElectronDiffraction(sample.inav[13:18,102:107])

#CORRECTING FOR CAMERA
dp.apply_affine_transformation(np.array([[0.99,0,0],
                                         [0,0.69,0],
                                         [0,0,1]]))
#CORRECTING FOR BEING OFF CENTER
dp.get_direct_beam_position(method='blur',sigma=10,progress_bar=False) #checking centers, also BUG!
dp.apply_affine_transformation(D=np.array([[1,0,0],
                                           [0,1,3],
                                           [0,0,1]]))

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

In [4]:
#dp.plot()

In [5]:
pixel_distance = 79.6# np.sqrt(50**2 + 61.7**2)

In [6]:
theoretical_distance = 1/np.sqrt((3.615**2)/(8))

In [7]:
dp.set_calibration(theoretical_distance/pixel_distance)

In [8]:
a = 3.615
B = pmg.Element("B")
N = pmg.Element("N")
lattice = pmg.Lattice.cubic(a)
structure = pmg.Structure.from_spacegroup("F-43m",lattice, [B,N], [[0, 0, 0],[0.25,0.25,0.25]])

In [9]:
def build_structure_lib(structure,rot_list):
    struc_lib = dict()
    struc_lib["A"] = (structure,rot_list)
    return struc_lib    

In [10]:
rot_array = np.loadtxt('/home/pc494/Documents/data/Rotation_Group/432',delimiter=',')

In [38]:
edc = pxm.DiffractionGenerator(300, 5e-2)
diff_gen = pxm.DiffractionLibraryGenerator(edc)
rot_list = list(np.round(rot_array,decimals=5))

In [39]:
len(rot_list)

624287

In [36]:
struc_lib = build_structure_lib(structure,rot_list)
library = diff_gen.get_diffraction_library(struc_lib,
                                           calibration=theoretical_distance/pixel_distance,
                                           reciprocal_radius=1.5,
                                           half_shape=(half_side_length,half_side_length),
                                           representation='euler',
                                           with_direct_beam=False)

In [30]:
indexer = IndexationGenerator(dp,library)
match_results = indexer.correlate()

A Jupyter Widget

In [37]:
library

{'A': {(-0.996,
   0.056460000000000003,
   -0.033009999999999998,
   -0.060920000000000002): {'Sim': <pyxem.signals.diffraction_simulation.DiffractionSimulation at 0x7f616da820b8>,
   'intensities': array([  2.17030604e-02,   5.97759430e+00,   1.18989987e-01,
            6.66340640e+00,   5.63758441e-01,   1.14753791e+01,
            5.95668093e-03,   1.85428009e-01,   2.55955509e-01,
            5.66968683e+00,   5.39563530e-01,   1.24703205e+01,
            1.36789196e-02,   6.28701940e-03,   5.36178507e+00,
            1.34424985e-01]),
   'pattern_norm': 20.711391417939865,
   'pixel_coords': array([[150, 323],
          [198, 352],
          [246, 381],
          [179, 275],
          [227, 304],
          [275, 333],
          [323, 362],
          [208, 227],
          [304, 285],
          [237, 179],
          [285, 208],
          [333, 237],
          [381, 266],
          [266, 131],
          [314, 160],
          [362, 189]])},
  (-0.99543000000000004,
   0.0514500000000

In [31]:
match_results.data

array([[[[ 0.        ,  0.9       ,  0.        , -0.3       ,  0.48901204],
         [ 0.        ,  0.9       , -0.        , -0.3       ,  0.48901204],
         [ 0.        ,  0.9       ,  0.        , -0.3       ,  0.48901204],
         [ 0.        ,  0.9       , -0.        , -0.3       ,  0.48901204],
         [ 0.        ,  0.9       , -0.        , -0.3       ,  0.48901204]],

        [[ 0.        ,  0.9       ,  0.        , -0.3       ,  0.48076345],
         [ 0.        ,  0.9       , -0.        , -0.3       ,  0.48076345],
         [ 0.        ,  0.9       ,  0.        , -0.3       ,  0.48076345],
         [ 0.        ,  0.9       , -0.        , -0.3       ,  0.48076345],
         [ 0.        ,  0.9       , -0.        , -0.3       ,  0.48076345]],

        [[ 0.        ,  0.9       ,  0.        , -0.3       ,  0.47077483],
         [ 0.        ,  0.9       , -0.        , -0.3       ,  0.47077483],
         [ 0.        ,  0.9       ,  0.        , -0.3       ,  0.47077483],
        

In [32]:
peaks = match_results.map(peaks_from_best_template,phase=["A"],library=library,inplace=False)
mmx,mmy = generate_marker_inputs_from_peaks(peaks)
dp.plot(cmap='viridis')
for mx,my in zip(mmx,mmy):
    m = hs.markers.point(x=mx,y=my,color='yellow',marker='x') #see visual test
    dp.add_marker(m,plot_marker=True,permanent=True)

A Jupyter Widget

KeyError: (0.90000000000000002, 0.0, -0.29999999999999999)

In [21]:
match_results.inav[0,0].data

array([[ 0.        ,  0.93801   , -0.00322   , -0.31315   ,  0.50598902],
       [ 0.        ,  0.9428    ,  0.01264   , -0.32945   ,  0.48255374],
       [ 0.        ,  0.89925   ,  0.01585   , -0.27938   ,  0.47867899],
       [ 0.        ,  0.92828   , -0.00955   , -0.32955   ,  0.47790108],
       [ 0.        ,  0.94118   , -0.01202   , -0.32947   ,  0.47741445]])

In [ ]:
p = library["A"][(26,180,40)]['Sim']
p.indices

In [ ]:
from transforms3d.euler import euler2mat

t = euler2mat(26,180,40, 'rzxz')
np.matmul(t,[0,0,1])

In [ ]:
len(rot_list)